# Predicting Return on Investment (ROI) of Consumer Loans

**Andrew Nicholls** | Email: andrew.s.nicholls@gmail.com | [Github](https://github.com/Booleans)

If you are viewing this notebook on Github I recommend using the following nbviewer link instead to ensure proper formatting and working interactive charts.

[nbviewer link](https://nbviewer.jupyter.org/github/Booleans/Lending-Club-Loan-Analysis/blob/master/Loan_Analysis_Regression.ipynb)

## Summary

**Contents:**

1. [Problem Definition and Background Information](#1)
2. [Data Preparation: Wrangling, Cleaning, and Feature Extraction](#2)
3. [Exploratory Data Analysis](#3)
4. [Machine Learning Models](#4)
5. [Results and Findings](#4)
<a id='1'></a>

# 1. Introduction

### Problem Definition

LendingClub Corporation operates as an online marketplace that connects borrowers and investors in the United States. Its marketplace facilitates various types of loan products for consumers and small businesses, including unsecured personal loans, super prime consumer loans, unsecured education and patient finance loans, and unsecured small business loans. The company also offers investors an opportunity to invest in a range of loans based on term and credit characteristics. However, many loans issued through Lending Club end up being defaulted on by the borrower. The goal of this notebook is to examine the historical loan data available in order to generate a model to predict the return on investment that a new loan will generate.

### Files Provided

Lending Club provides CSV files of historical data for its loans. These files contain complete loan data for all loans issued through the time period stated, including the loan status (Current, Late, Fully Paid, etc.) and latest payment information. The files can be acquired on the [Lending Club Statistics Page](https://www.lendingclub.com/info/download-data.action). As of the creation of this project the latest data available from Lending Club was for Q1 2018.

For information on the definitions of fields contained within the historical data please see the [Lending Club Data Dictionary](https://github.com/Booleans/consumer-loan-survival-analysis/blob/master/data/LCDataDictionary.xlsx?raw=true).

The LoanStats3a file also contains information on loan applications that were declined and never issued. I have discarded those rows of data as they are not relevant to predicting loan defaults. I have also removed the last 2 rows of every spreadsheet, as they contained aggregate information on the number of loans in the file.

In [6]:
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import plotly
import seaborn as sns

<a id='2'></a>


# 2. Data Preparation: Wrangling, Cleaning, and Feature Extraction

Certain columns may be immediately ignored and not loaded into the dataframe. Some columns provide information that was not available upon loan origination (e.g. interest received from loan payments, late fees collected). Other columns are dropped for containing information that is not relevant to whether a loan will default or not (e.g. borrower id number, url that the loan is listed at). Joint loan applications are relatively new at Lending Club therefore all columns related to joint applications and secondary applicants will be ignored.

For a complete list of columns ignored please see the [columns.py](Column_Info.py) file.

### Create DataFrame of Relevant Loans:

We only want loans that have either been fully paid or charged off. We cannot calculate the final return on investment of loans that are still current. 

In [34]:
%run src/columns.py

loan_data = []
csv_file_names = ('LoanStats3a_securev1', 'LoanStats3b_securev1', 'LoanStats3c_securev1', 'LoanStats3d_securev1',
                  'LoanStats_securev1_2016Q1', 'LoanStats_securev1_2016Q2', 'LoanStats_securev1_2016Q3',
                  'LoanStats_securev1_2016Q4', 'LoanStats_securev1_2017Q1', 'LoanStats_securev1_2017Q2', 
                  'LoanStats_securev1_2017Q3', 'LoanStats_securev1_2017Q4', 'LoanStats_securev1_2018Q1')

for file in csv_file_names:    
    data = pd.read_csv('data/' + file + '.csv', header = 1, low_memory=False, na_values='n/a', usecols=columns_to_use, nrows=1000) 
    loan_data.append(data)
    
loans = pd.concat(loan_data)

# We only want loans that have completed. Loan status should be either Fully Paid or Charged Off.
loans = loans[(loans['loan_status'] == 'Charged Off') | (loans['loan_status'] == 'Fully Paid')]
loans.drop('loan_status', axis=1, inplace=True)

# As stated previously, we are ignoring loans with a joint applicant.
loans = loans[loans['application_type'] == "INDIVIDUAL"]
loans.drop(labels=['application_type'], axis=1, inplace=True)

print(str(loans.shape[0]) + ' loans with ' + str(loans.shape[1]) + ' features.')

3415 loans with 88 features.


In [37]:
loans.tail(20)

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
244,12000,60 months,28.69%,378.65,F,4 years,MORTGAGE,44460.0,Not Verified,Mar-17,...,0.0,2.0,81.5,100.0,0,0,204439.0,57585.0,7700.0,57939.0
395,7450,36 months,12.74%,250.09,C,2 years,MORTGAGE,99599.0,Verified,Mar-17,...,0.0,4.0,100.0,33.3,1,0,776589.0,61667.0,45100.0,42500.0
405,3000,36 months,11.49%,98.92,B,NaN,RENT,25000.0,Source Verified,Mar-17,...,0.0,4.0,96.6,0.0,2,0,28911.0,6392.0,17200.0,7611.0
474,2000,36 months,8.24%,62.90,B,< 1 year,MORTGAGE,85816.0,Source Verified,Mar-17,...,0.0,1.0,92.0,100.0,0,0,257598.0,78393.0,23100.0,57382.0
486,8000,36 months,13.99%,273.39,C,NaN,OWN,72000.0,Not Verified,Mar-17,...,0.0,3.0,94.7,0.0,1,0,45039.0,33030.0,7000.0,30539.0
496,5000,36 months,11.39%,164.62,B,4 years,MORTGAGE,50000.0,Source Verified,Mar-17,...,0.0,0.0,76.9,100.0,0,0,11500.0,7540.0,4000.0,6500.0
505,25000,60 months,29.99%,808.69,F,10+ years,RENT,66560.0,Source Verified,Mar-17,...,0.0,2.0,100.0,100.0,1,0,55597.0,36955.0,4300.0,40597.0
530,10000,36 months,12.74%,335.69,C,10+ years,OWN,32425.0,Source Verified,Mar-17,...,0.0,1.0,100.0,0.0,0,0,57672.0,24614.0,35400.0,22272.0
531,25000,60 months,18.99%,648.38,D,2 years,MORTGAGE,80000.0,Verified,Mar-17,...,0.0,9.0,100.0,0.0,0,0,304139.0,26438.0,30400.0,22069.0
538,8000,36 months,15.99%,281.22,C,3 years,RENT,37000.0,Source Verified,Mar-17,...,0.0,0.0,100.0,100.0,0,0,35874.0,32168.0,19700.0,16174.0


### Cleaning the Data



In [29]:
# Columns that end with % were read in as strings instead of floats. We need to remove the % and change data types.
rate_cols = ['int_rate', 'revol_util']

for col in rate_cols:
    loans[col] = loans[col].str.rstrip('%').astype('float64')
all_col_names = list(loans.columns)

# issue_d column can be in two formats. This function will handle the conversion of both formats.
def convert_date(col_date):
    if col_date[0].isdigit():
        # Need to pad the date string with a 0 if it's too short. 
        col_date = col_date.rjust(6, '0')
        return pd.to_datetime(col_date, format = '%y-%b')
    else:
        try:
            return pd.to_datetime(col_date, format = '%b-%y')
        except:
            return pd.to_datetime(col_date, format = '%b-%Y')
    
loans['issue_d'] = loans['issue_d'].map(convert_date)
loans['year'] = loans.issue_d.dt.year
loans['month'] = loans.issue_d.dt.month

loans['has_recent_delinq'] = loans['mths_since_last_delinq'].notnull().astype(int)
loans['has_recent_revol_delinq'] = loans['mths_since_recent_revol_delinq'].notnull().astype(int)

# Term should be either '36' or '60'.
loans['term'] = ['36' if row.strip() == '36 months' else '60' for row in loans['term']]

loans['emp_length'] = [0 if row == '< 1 year' else row for row in loans['emp_length']]
loans['emp_length'].fillna(int(0), inplace=True)
loans['emp_length'] = loans['emp_length'].str.extract('(\d+)', expand=True).astype(float)
loans['emp_length'].fillna(int(0), inplace=True)

In [33]:
loans

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,5000,36 months,10.65,162.87,B,10+ years,RENT,24000.00,Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
1,2500,60 months,15.27,59.83,C,< 1 year,RENT,30000.00,Source Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2400,36 months,15.96,84.33,C,10+ years,RENT,12252.00,Not Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,10000,36 months,13.49,339.31,C,10+ years,RENT,49200.00,Source Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,3000,60 months,12.69,67.79,B,1 year,RENT,80000.00,Source Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
5,5000,36 months,7.90,156.46,A,3 years,RENT,36000.00,Source Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
6,7000,60 months,15.96,170.08,C,8 years,RENT,47004.00,Not Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
7,3000,36 months,18.64,109.43,E,9 years,RENT,48000.00,Source Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
8,5600,60 months,21.28,152.39,F,4 years,OWN,40000.00,Source Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
9,5375,60 months,12.69,121.45,B,< 1 year,RENT,15000.00,Verified,Dec-11,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
